In [ ]:
from PyQt5.QtWidgets import *
import sys
import cv2 as cv
import winsound
import numpy as np

In [ ]:
class BeepSound(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('삑')
        self.setGeometry(200,200,500,100)

        shortBeepButton=QPushButton('짧게',self)
        longBeepButton=QPushButton('길게',self)
        quitButton=QPushButton('나가기',self)
        self.label=QLabel('환영함',self)

        shortBeepButton.setGeometry(10,10,100,30)
        longBeepButton.setGeometry(110,10,100,30)
        quitButton.setGeometry(210,10,100,30)
        self.label.setGeometry(10,40,500,70)

        shortBeepButton.clicked.connect(self.shortBeepFunction)
        longBeepButton.clicked.connect(self.longBeepFunction)
        quitButton.clicked.connect(self.quitFunction)
    
    def shortBeepFunction(self):
        self.label.setText('주파수 1000으로 0.5초 동안 삑소리')
        winsound.Beep(1000,500)

    def longBeepFunction(self):
        self.label.setText('주파수 1000으로 3초 동안 삑소리')
        winsound.Beep(1000,3000)

    def quitFunction(self):
        self.close()

app=QApplication(sys.argv)
win=BeepSound()
win.show()
app.exec_()

In [ ]:
class Video(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('비디오에서 프레임 수정')
        self.setGeometry(200,200,500,100)

        videoButton=QPushButton('비디오 켜기',self)
        captureButton=QPushButton('프레임 잡기',self)
        saveButton=QPushButton('저장',self)
        quitButton=QPushButton('나가기',self)

        videoButton.setGeometry(10,10,100,30)
        captureButton.setGeometry(110,10,100,30)
        saveButton.setGeometry(210,10,100,30)
        quitButton.setGeometry(310,10,100,30)

        videoButton.clicked.connect(self.videoFunction)
        captureButton.clicked.connect(self.captureFunction)
        saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)


    def videoFunction(self):
        self.cap=cv.VideoCapture(0,cv.CAP_DSHOW)
        if not self.cap.isOpened(): self.close()

        while True:
            ret,self.frame=self.cap.read()
            if not ret: break
            cv.imshow('video display',self.frame)
            cv.waitKey(1)
    
    def captureFunction(self):
        self.captureFrame=self.frame
        cv.imshow('Captured Frame',self.captureFrame)
    
    def saveFunction(self):
        fname=QFileDialog.getSaveFileName(self,'파일저장','./')
        cv.imwrite(fname[0],self.capturedFrame)
    
    def quitFunction(self):
        self.cap.release()
        cv.destroyAllWindows()
        self.close()

app=QApplication(sys.argv)
win=Video()
win.show()
app.exec_()

In [ ]:
class Orim(QMainWindow):
    def __init__(self) :
        super().__init__()
        self.setWindowTitle('오림')
        self.setGeometry(200,200,700,200)

        fileButton=QPushButton('파일',self)
        paintButton=QPushButton('그림림',self)
        cutButton=QPushButton('자르기기',self)
        incButton=QPushButton('+',self)
        decButton=QPushButton('-',self)
        saveButton=QPushButton('저장장',self)
        quitButton=QPushButton('나기기',self)

        fileButton.setGeometry(10,10,100,30)
        paintButton.setGeometry(110,10,100,30)
        cutButton.setGeometry(210,10,100,30)
        incButton.setGeometry(310,10,50,30)
        decButton.setGeometry(360,10,50,30)
        saveButton.setGeometry(410,10,100,30)
        quitButton.setGeometry(510,10,100,30)

        fileButton.clicked.connect(self.fileOpenFunction)
        paintButton.clicked.connect(self.paintFunction)
        cutButton.clicked.connect(self.cutFunction)
        incButton.clicked.connect(self.incFunction)
        decButton.clicked.connect(self.decFunction)
        saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)

        self.BrushSiz=5
        self.LColor,self.RColor=(255,0,0),(0,0,255)

    def fileOpenFunction(self):
        fname=QFileDialog.getOpenFileName(self,'Open file>','./')
        self.img=cv.imread(fname[0])
        if self.img is None: sys.exit('파일을 못찾았으')

        self.img_show=np.copy(self.img)
        cv.imshow('Painting',self.img_show)

        self.mask=np.zeros((self.img.shape[0],self.img.shape[1]),np.uint8)
        self.mask[:,:]=cv.GC_PR_BGD

    def paintFunction(self):
        cv.setMouseCallback('Paint',self.painting)

    def painting(self,event,x,y,flags,param):
        if event==cv.EVENT_LBUTTONDOWN:
            cv.circle(self.img_show,(x,y),self.BrushSiz,self.LColor,-1)
            cv.circle(self.mask,(x,y),self.BrushSiz,cv.GC_FGD,-1)
        elif event==cv.EVENT_RBUTTONDOWN:
            cv.circle(self.img_show,(x,y),self.BrushSiz,self.RColor,-1)
            cv.circle(self.mask,(x,y),self.BrushSiz,cv.GC_BGD,-1)
        elif event==cv.EVENT_MOUSEMOVE and flags==cv.EVENT_FLAG_LBUTTON:
            cv.circle(self.img_show,(x,y),self.BrushSiz,self.LColor)
            cv.circle(self.mask,(x,y),self.BrushSiz,cv.GC_FGD,-1)
        elif event==cv.EVENT_MOUSEMOVE and flags==cv.EVENT_FLAG_RBUTTON:
            cv.circle(self.img_show,(x,y),self.BrushSiz,self.RColor)
            cv.circle(self.mask,(x,y),self.BrushSiz,cv.GC_BGD,-1)
    
        cv.imshow('Painting',self.img_show)

    def cutFunction(self):
        background=np.zeros((1,65),np.float64)
        foreground=np.zeros((1,65),np.float64)
        cv.grabCut(self.img,self.mask,None,background,foreground,5,cv.GC_INIT_WITH_MASK)
        mask2=np.where((self.mask==2)|(self.mask==0),0,1).astype('uint8')
        self.grabImg=self.img*mask2[:,:,np.newaxis]
        cv.imshow('Scissoring',self.grabImg)

    def incFunction(self):
        self.BrushSiz=min(20,self.BrushSiz+1)

    def decFunction(self):
        self.BrushSiz=max(1,self.BrushSiz-1)

    def saveFunction(self):
        fname=QFileDialog.getSaveFileName(self,'파일 저장','./')
        cv.imwrite(fname[0],self.grabImg)

    def quitFunction(self):
        cv.destroyAllWindows()
        self.close()

app=QApplication(sys.argv)
win=Orim()
win.show()
app.exec_()



In [ ]:
class TrafficWeak(QMainWindow):
    def __init__(self):
        super().__init__()
        # 윈도우 제목,위치,크기 설정
        self.setWindowTitle('교통약자 보호')
        self.setGeometry(200,200,700,200)
       
        # 버튼 4개, 레이블 1개 생성
        signButton=QPushButton('표지판 등록',self)
        roadButton=QPushButton('도로 영상 불러옴',self)
        recognitionButton=QPushButton('인식',self)
        quitButton=QPushButton('나가기',self)
        self.label=QLabel('환영합니다!',self)
        
        # 버튼 클릭 시 수행할 콜백 함수 지정
        signButton.setGeometry(10,10,100,30)
        roadButton.setGeometry(110,10,100,30)
        recognitionButton.setGeometry(210,10,100,30)
        quitButton.setGeometry(510,10,100,30)
        self.label.setGeometry(10,40,600,170)
        
        signButton.clicked.connect(self.signFunction)
        roadButton.clicked.connect(self.roadFunction) 
        recognitionButton.clicked.connect(self.recognitionFunction)        
        quitButton.clicked.connect(self.quitFunction)

        self.signFiles=[['child.png','어린이'],['elder.png','노인'],['disabled.png','장애인']]	# 표지판 모델 영상 이름 설정
        self.signImgs=[]				# 표지판 모델 영상 저장할 signImgs 객체 생성

    # 콜백함수 선언     
    def signFunction(self):
        self.label.clear()  # 레이블 지움
        self.label.setText('교통약자 번호판을 등록합니다.') # 메시지 출력
        
        for fname,_ in self.signFiles:  # signFiles 리스트에 들어있는 요소 각각에 대해 파일 이름을 fname에 담음
            self.signImgs.append(cv.imread(fname))  # 이름이 fname인 영상 파일 읽어 signImgs에 추가
            cv.imshow(fname,self.signImgs[-1])      # signImgs의 맨 뒤에 있는(= 방금 추가한 영상)을 fname이라는 제목 표시줄 가진 윈도우에 디스플레이

    def roadFunction(self):
        if self.signImgs==[]: # signImgs가 비었을 때(= 사용자가 표지판 등록 안 하고 '도로 영상 불러옴' 버튼 클릭했을 때)
            self.label.setText('먼저 번호판을 등록하세요.') # 메시지 표시
        else:   # 표지판 제대로 등록해둔 경우
            fname=QFileDialog.getOpenFileName(self,'파일 읽기','./')    # 사용자가 폴더 브라우징하면서 도로 영상 선택할 수 있게 함
            self.roadImg=cv.imread(fname[0])    # 영상 파일 읽어 roadImg 객체에 저장
            if self.roadImg is None: sys.exit('파일을 찾을 수 없습니다.')  
    
            cv.imshow('Road scene',self.roadImg)  # 읽은 영상을 윈도우에 디스플레이
        
    def recognitionFunction(self):
        if self.roadImg is None:    # 사용자가 도로 영상 입력 안 하고 '인식' 버튼 클릭했을 때
            self.label.setText('먼저 도로 영상을 입력하세요.')
        else:   # 도로 영상 제대로 입력해둔 경우
            sift=cv.SIFT_create()   # SIFT 검출하는 데 사용할 객체 생성

            ## signImgs에 담겨있는 3장의 표지판 모델 영상에서 특징점 & 기술자 차례로 추출하여 KD에 저장
            KD=[] # 여러 표지판 영상의 키포인트와 기술자 저장
            for img in self.signImgs: 
                gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
                KD.append(sift.detectAndCompute(gray,None))
                
            grayRoad=cv.cvtColor(self.roadImg,cv.COLOR_BGR2GRAY) # 명암으로 변환
            road_kp,road_des=sift.detectAndCompute(grayRoad,None) # 키포인트와 기술자 추출
            
            ## 여러 표지판 영상의 good match를 저장 => GD : 표지판 영상 3장 각각에 대해 좋은 매칭 리스트 가지게 됨  
            matcher=cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)    # FLANN 기반 매칭해주는 matcher 객체 생성
            GM=[]   # 3장의 모델 영상이랑 매칭한 결과 저장할 GM 객체 생성
            for sign_kp,sign_des in KD: # 표지판 모델 영상에서 추출해둔 특징점 & 기술자를 KD에서 하나씩 꺼내서 sign_kp, sign_des에 저장하고 반복문 실행
                knn_match=matcher.knnMatch(sign_des,road_des,2) # sign_des & road_des 매칭하여 특징점마다 최근접 이웃 2개 찾음
                T=0.7
                # 좋은 매칭 골라 good_match에 저장
                good_match=[]
                for nearest1,nearest2 in knn_match:
                    if (nearest1.distance/nearest2.distance)<T:
                        good_match.append(nearest1)
                GM.append(good_match) # good_match를 GD에 추가       
            
            best=GM.index(max(GM,key=len)) # 표지판 영상 3장의 매칭 리스트 중 max 찾음 => 매칭 쌍 개수가 최대인 번호판 찾기
            
            if len(GM[best])<4:	# 최선의 번호판이 매칭 쌍 4개 미만이면 인식 실패로 간주
                self.label.setText('표지판이 없습니다.')  
            else:			# 인식에 성공한 경우(호모그래피 찾아 영상에 표시)
                sign_kp=KD[best][0] # 표지판 영상의 특징점
                good_match=GM[best] # 매칭 쌍 정보 저장
            
                points1=np.float32([sign_kp[gm.queryIdx].pt for gm in good_match])
                points2=np.float32([road_kp[gm.trainIdx].pt for gm in good_match])
                
                H,_=cv.findHomography(points1,points2,cv.RANSAC)
                
                h1,w1=self.signImgs[best].shape[0],self.signImgs[best].shape[1] # 번호판 영상의 크기
                h2,w2=self.roadImg.shape[0],self.roadImg.shape[1] # 도로 영상의 크기
                
                box1=np.float32([[0,0],[0,h1-1],[w1-1,h1-1],[w1-1,0]]).reshape(4,1,2)
                box2=cv.perspectiveTransform(box1,H)
                
                self.roadImg=cv.polylines(self.roadImg,[np.int32(box2)],True,(0,255,0),4)
                
                img_match=np.empty((max(h1,h2),w1+w2,3),dtype=np.uint8)
                cv.drawMatches(self.signImgs[best],sign_kp,self.roadImg,road_kp,good_match,img_match,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
                cv.imshow('Matches and Homography',img_match)
                
                self.label.setText(self.signFiles[best][1]+' 보호구역입니다. 30km로 서행하세요.')   # best가 0:어린이, 1:노인, 2:장애인                 
                winsound.Beep(3000,500) # 비프음 발생         
                      
    def quitFunction(self): # '나가기' 버튼 클릭 시 실행
        cv.destroyAllWindows()        
        self.close()
                
app=QApplication(sys.argv) 
win=TrafficWeak() 
win.show()
app.exec_()